In [1]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

import pandas, seaborn, numpy, collections, sefara, varcode, uncertainties, numpy, statsmodels, pickle, scipy
%matplotlib inline
from matplotlib import pyplot
import topiary
import mhctools
import matplotlib_venn
import intervaltree
import statsmodels.robust
import mhcflurry
from varcode.common import memoize
pandas.set_option('display.max_columns', None)
import scipy, sys, sklearn
import sklearn.utils
import matplotlib
import common as c
import sklearn.tree
import pymc3
from scipy import optimize
import glob


def print_full(x):
    pandas.set_option('display.max_rows', len(x))
    print(x)
    pandas.reset_option('display.max_rows')

#matplotlib.rc("savefig", dpi=800)
matplotlib.rc("savefig", dpi=72)
matplotlib.rc('text', usetex=False)
#reload(c)

%matplotlib inline
%config InlineBackend.figure_format = 'png'

import pystan
import pyfaidx



/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


Couldn't import dot_parser, loading of dot files will not be possible.
Using mutation cache: ../data/derived/mutations.varlens.all.csv.bz2._cached_extra_annotations.csv


Using Theano backend.


In [2]:
df = pandas.read_csv("../data/original/c_elegans_substitutions.csv")
df["context_mutation_3p5p"] = df.CONTEXT.str.slice(2, 3) + "(" + df.MUT_TYPE + ")" + df.CONTEXT.str.slice(4, 5)
df

,ID,Genotype,Drug,Concentration,CHR,POS,REF_BASE,TOP_GENOTYPE,TOP_GENOTYPE_PROB,2ND_GENOTYPE,2ND_GENOTYPE_PROB,NORM_A,NORM_C,NORM_G,NORM_T,MUT_A,MUT_C,MUT_G,MUT_T,NORM_COVERAGE,TUM_COVERAGE,MUT_TYPE,CONTEXT,GENE_FOOTPRINT,STRAND_TX,context_mutation_3p5p
0,CD0001a,N2,None,0,I,6945674,A,AA/GG,1.00,AG/GG,3.800000e-12,46,0,0,0,0,0,60,0,46,60,T>C,CTTTAAA,No_gene,No_gene,T(T>C)A
1,CD0001a,N2,None,0,II,12649008,T,TT/AT,1.00,AT/AT,1.600000e-09,0,0,0,37,25,0,0,17,37,42,T>A,TTTTAAA,No_gene,No_gene,T(T>A)A
2,CD0001a,N2,None,0,II,12770220,T,TT/GG,0.81,TT/GT,1.900000e-01,0,0,0,21,0,0,2,0,21,2,T>G,CAATACG,Y46G5A.13,Non-transcribed,A(T>G)A
3,CD0001a,N2,None,0,II,13334311,G,GG/AA,0.90,GG/AG,1.000000e-01,0,0,43,0,3,0,0,0,43,3,C>T,CTGCGTC,No_gene,No_gene,G(C>T)G
4,CD0001a,N2,None,0,II,13334507,A,AA/AG,1.00,AG/AG,8.800000e-08,31,0,0,0,15,0,6,0,31,21,T>C,TGGTGCC,No_gene,No_gene,G(T>C)G
5,CD0001a,N2,None,0,II,13334511,C,CC/AC,1.00,AC/AC,2.300000e-08,0,33,0,0,6,15,0,0,33,21,C>A,CCACTTG,No_gene,No_gene,A(C>A)T
6,CD0001a,N2,None,0,II,13334512,T,TT/GT,1.00,GT/GT,2.200000e-08,0,0,0,33,0,0,6,15,33,21,T>G,CACTTGC,No_gene,No_gene,C(T>G)T
7,CD0001a,N2,None,0,II,13334523,A,AA/AG,1.00,AG/AG,3.100000e-09,36,0,0,0,14,0,14,0,36,28,T>C,ACGTAGA,No_gene,No_gene,G(T>C)A
8,CD0001a,N2,None,0,II,13334540,T,TT/CT,1.00,CT/CT,2.100000e-10,0,0,0,40,0,17,0,10,40,27,T>C,AAATGCG,No_gene,No_gene,A(T>C)G
9,CD0001a,N2,None,0,V,18982871,C,CC/CT,1.00,CT/CT,5.700000e-11,0,42,0,0,0,29,0,20,42,49,C>T,CTGCTAA,Y39B6A.39,Non-transcribed,G(C>T)T


In [3]:
def sort_key(context):
    return (context.split(">")[0][-1],context.split(">")[1][0], context  )
all_contexts = sorted(set(df.context_mutation_3p5p), key=sort_key)


In [4]:
def normalize(series):
    return series / series.sum()


In [5]:
df.applymap?

In [6]:
#counts_dosed = df[(df.Drug == "Cisplatin") & (df.Concentration > 0) ].groupby(["context_mutation_3p5p", "Genotype", ]).CONTEXT.count().unstack().ix[all_contexts].fillna(0)
#counts_undosed = df[(df.Concentration == 0) ].groupby(["context_mutation_3p5p", "Genotype", ]).CONTEXT.count().unstack().ix[all_contexts].fillna(0)
#contexts_to_use = counts_dosed / (counts_undosed + .01) > 2
#contexts_to_use = contexts_to_use.ix[contexts_to_use].index
#print(contexts_to_use)

#fractions = normalize((counts_dosed - counts_undosed).dropna(axis=1).applymap(lambda x: max(x,0)).sum(axis=1))
#counts_dosed

In [ ]:
'''
cisplatin_df = df[df.ID.isin(dosed_wildtype_ids)].copy()
cisplatin_df = df[(df.Drug == "Cisplatin") & (df.Concentration > 0) ].copy()

assert cisplatin_df.Drug.unique() == ["Cisplatin"]

cisplatin_df.shape

counts = cisplatin_df.context_mutation_3p5p.value_counts()
fractions = counts / counts.sum()
'''

'\ncisplatin_df = df[df.ID.isin(dosed_wildtype_ids)].copy()\ncisplatin_df = df[(df.Drug == "Cisplatin") & (df.Concentration > 0) ].copy()\n\nassert cisplatin_df.Drug.unique() == ["Cisplatin"]\n\ncisplatin_df.shape\n\ncounts = cisplatin_df.context_mutation_3p5p.value_counts()\nfractions = counts / counts.sum()\n'

In [ ]:
# C elegans genome for normalization constants
# THIS IS SLOW
import Bio, gzip
c_elegans_seq = Bio.SeqIO.parse(gzip.open("../data/original/Caenorhabditis_elegans.WBcel235.dna_sm.toplevel.fa.gz"), 'fasta')
c_elegans_seq

counts = collections.Counter()
for record in c_elegans_seq:
    for i in range(len(record.seq) - 3):
        if (i % 100000 == 0):
            print("%d / %d = %f%%" % (i, len(record.seq), i * 100.0 / len(record.seq)))
        sub = str(record.seq[i:i+3])
        if sub == sub.upper():
            counts[sub] += 1
            
def canonicalize(trinucleotide):
    (left, middle, right) = trinucleotide
    if middle in ('A', 'G'):
        return pyfaidx.complement(right) + pyfaidx.complement(middle) + pyfaidx.complement(left)
    return trinucleotide

canonicalized_counts = [(canonicalize(key), count) for (key, count) in counts.items()]
genome_counts = pandas.DataFrame(
    canonicalized_counts, columns=["context", "count"]).groupby("context")["count"].sum().to_frame()
genome_counts

0 / 15072434 = 0.000000%
100000 / 15072434 = 0.663463%
200000 / 15072434 = 1.326926%
300000 / 15072434 = 1.990389%
400000 / 15072434 = 2.653851%
500000 / 15072434 = 3.317314%
600000 / 15072434 = 3.980777%
700000 / 15072434 = 4.644240%
800000 / 15072434 = 5.307703%
900000 / 15072434 = 5.971166%
1000000 / 15072434 = 6.634628%
1100000 / 15072434 = 7.298091%
1200000 / 15072434 = 7.961554%
1300000 / 15072434 = 8.625017%
1400000 / 15072434 = 9.288480%
1500000 / 15072434 = 9.951943%
1600000 / 15072434 = 10.615406%
1700000 / 15072434 = 11.278868%
1800000 / 15072434 = 11.942331%
1900000 / 15072434 = 12.605794%
2000000 / 15072434 = 13.269257%
2100000 / 15072434 = 13.932720%
2200000 / 15072434 = 14.596183%
2300000 / 15072434 = 15.259646%
2400000 / 15072434 = 15.923108%
2500000 / 15072434 = 16.586571%
2600000 / 15072434 = 17.250034%
2700000 / 15072434 = 17.913497%
2800000 / 15072434 = 18.576960%
2900000 / 15072434 = 19.240423%
3000000 / 15072434 = 19.903885%
3100000 / 15072434 = 20.567348%
3200000

In [ ]:
def normalize_by_counts(sub_df):
    sub_df = sub_df.to_frame()
    sub_df.columns = ['count']
    sub_df["normalization"] = [x[0] + x[2] + x[-1] for x in sub_df.index]
    sub_df["normalization_value"] = [genome_counts.ix[x]["count"] for x in sub_df.normalization]
    sub_df["raw_fraction"] = sub_df["count"] / sub_df["normalization_value"]
    sub_df["fully_normalized_fraction"] = normalize(sub_df["count"] / sub_df["normalization_value"])
    print(sub_df)
    return sub_df.fully_normalized_fraction

fractions = normalize_by_counts(
    df[(df.Genotype == "N2") & (df.Drug == "Cisplatin") & (df.Concentration > 0) ].context_mutation_3p5p.value_counts()[all_contexts].fillna(0)
)
fractions

In [ ]:
df

In [ ]:
signatures_df = pandas.read_table("../data/original/signatures_probabilities.txt")
signatures_df["Somatic Mutation Type"] = signatures_df["Somatic Mutation Type"].str.replace("[", "(").str.replace("]", ")")
signatures_df.index = signatures_df["Somatic Mutation Type"]
for column in signatures_df.columns:
    if 'signature' not in column.lower():
        del signatures_df[column]
signatures_df #.sum(axis=0)

In [ ]:
signatures_df["Cisplatin"] = fractions
signatures_df["Cisplatin"] = signatures_df["Cisplatin"].fillna(0)

def sort_key(context):
    return (context.split(">")[0][-1],context.split(">")[1][0], context  )
signatures_df = signatures_df.ix[sorted(signatures_df.index, key=sort_key)]


signatures_df.to_csv("../data/derived/all_signatures.csv")
signatures_for_deconstructsigs_df = signatures_df.T.copy()
signatures_for_deconstructsigs_df.columns = [
    x.replace("(", "[").replace(")", "]") for x in signatures_for_deconstructsigs_df.columns
]
signatures_for_deconstructsigs_df.to_csv("../data/derived/all_signatures_for_deconstructsigs.csv")

signatures_df #.sum(axis=0)

In [ ]:
seaborn.set_context('poster')
pyplot.figure(figsize=(30,10))
seaborn.barplot(data=signatures_df.reset_index(), y="Cisplatin", x="Somatic Mutation Type")
pyplot.xticks(rotation=90)
pyplot.ylabel("probability")
pyplot.title("Cisplatin signature")